# Deconvolution Problems 

We consider the two-dimensional deconvolution problems to find a non-negative function f given data 
$$
    d \sim \mathrm{Pois}(h*f)
$$
with a non-negative convolution kernel $h$, and $\mathrm{Pois}$ denotes the element-wise Poisson distribution.


In [ ]:
import sys
from pathlib import Path

import numpy as np

from regpy.operators import GaussianBlur
from regpy.solvers import Setting
from regpy.solvers.linear import TikhonovCG
from regpy.functionals import *
from regpy.hilbert import L2


example_dir = "../../../../examples/deconvolution"
Path(example_dir).resolve()
sys.path.insert(0, str(example_dir))
from plots import comparison_plot, convergence_plot, plot_recos
from test_images import mixed


## Generate synthetic data with impulsive noise

In [ ]:
grid, exact_sol = mixed(M=256,N=256,c_bubbles=1.,fac=50.)
r"""grid is the underlying UniformGridFcts vector space, and exact_sol the exact solution."""
a=0.05
conv =  GaussianBlur(grid,a,pad_amount=16)
r"""Convolution operator \(f\mapsto h*f\) for the convolution kernel \(h(x)=\exp(-|x|_2^2/a^2)\)."""
blur = conv(exact_sol)
blur[blur<0] = 0.
data = blur.copy()
n,m=grid.shape
for j in range(64**2):
    nn = np.random.randint(n)
    mm = np.random.randint(m)
    data[nn,mm] =  np.random.randint(2000)-1000

comparison_plot(grid,exact_sol,data,title_left='noisy measurement data')

## Reconstruction with a quadratic data fidelity term
... fails completely for this data!

In [ ]:
alpha = 1e-4
setting = Setting(conv,L2,L2)
solver = TikhonovCG(setting=setting, data = data, regpar = alpha,reltolx=1e-6)
fal,_ = solver.run()

comparison_plot(grid,exact_sol,fal,title_left='quadratic Tikhonov regularization')

## setting with Huber data fidelity and quadratic penalty term
We now set up a generalized Tikhonov regularization setting
$$
\hat{f} \in \mathrm{argmin}_f\left[\frac{1}{\alpha}H_{\sigma}(Tf-g^{\mathrm{obs}})+\|f\|^2\right]    
$$
where the Huber data fidelity term $H_{\sigma}$ is quadratic on coordinates in $[-\sigma,\sigma]$ and $|\cdot|-\sigma/2$ for large coordinates.



In [ ]:
from regpy.functionals import Huber,LppPower
sigma = 0.1
Sdat = (1./sigma)*Huber(grid,sigma=sigma,eps =1e-10)
L2pen = LppPower(grid,p=2)
huber_setting = Setting(conv,L2pen,Sdat,alpha,data=data)

display the available methods

In [ ]:
huber_setting.display_all_methods(full_names=False)

In [ ]:
methods =  ['FISTA','dual_FISTA','PDHG','dual_PDHG','dual_FB']

recos = {}
for method in methods:
    #huber_setting.set_stopping_rule(method, OptimalityCondStopping(huber_setting,tol=0.01,max_iter=500,logging_level=logging.INFO))
    recos[method]= huber_setting.run(method)[0]

plot the convergence histories 

In [ ]:
convergence_plot(huber_setting,methods,'Huber data fid., quadratic penalty')

visualize the reconstructions

In [ ]:
plot_recos(huber_setting,methods,recos,truth=exact_sol)